In [20]:
import os
import urllib.request
from urllib.parse import urlparse
import requests

In [4]:

#os.chdir("../")

In [6]:
from TGSRTC_Productivity.constants import *
from TGSRTC_Productivity.utils.common import read_yaml, create_directories

In [7]:
%pwd

'c:\\Users\\sr\\OneDrive\\Documents\\Work\\Agni\\Development\\Python\\TGSRTC_Productivity'

In [10]:
from dataclasses import dataclass
from pathlib import Path

# Maps to the structure in the config/config.yaml file
# Its the return type of the function

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [10]:
print("Current Working Directory:", os.getcwd())

Current Working Directory: c:\Users\sr\OneDrive\Documents\Work\Agni\Development\Python\TGSRTC_Productivity


In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [11]:
import os
import urllib.request as request
import zipfile
from TGSRTC_Productivity import logger
from TGSRTC_Productivity.utils.common import get_size

In [22]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        try:
            # Check if the file already exists
            if not os.path.exists(self.config.local_data_file):
                # Attempt to download the file

                #filename, headers = urllib.request.urlretrieve(
                    #url=self.config.source_URL,
                    #filename=self.config.local_data_file
                #)
                url=self.config.source_URL
                local_filename = self.config.local_data_file
                with requests.get(url, stream=True) as r:
                    r.raise_for_status()  # Check for errors
                    with open(local_filename, 'wb') as f:
                        for chunk in r.iter_content(chunk_size=8192):
                            if chunk:  # Filter out keep-alive new chunks
                                f.write(chunk)
          
                logger.info(f"downloaded successfully! Info:")
            else:
                # Log that the file already exists and display its size
                file_size = get_size(Path(self.config.local_data_file))
                print(f"File already exists. Size: {file_size}")
                logger.info(f"File already exists with size: {file_size}")

        except Exception as e:
            # Handle exceptions (e.g., HTTP errors, file errors)
            print(f"An error occurred during file download: {str(e)}")
            logger.error(f"Error during download: {str(e)}")
            raise



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [26]:
try:

    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-09-23 23:03:20,316: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-23 23:03:20,321: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-23 23:03:20,325: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-09-23 23:03:20,327: INFO: common: created directory at: artifacts]
[2024-09-23 23:03:20,329: INFO: common: created directory at: artifacts/data_ingestion]
[2024-09-23 23:03:21,227: INFO: 282925895: downloaded successfully! Info:]
